In [ ]:
# !pip install geojson
# ! pip install rasterio

In [ ]:
import ee
import pandas as pd
import json
from glob import glob
import geemap
import pyproj
from geopandas import geopandas as gpd
from shapely.geometry import Point
from copy import deepcopy
import numpy as np
import os
import ast

# import geojson

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ee.Authenticate()
# ee.Initialize(project='ee-mtpictd')

# Merge CSVs

In [ ]:
best_month_dict = {'Eastern Plateau & Hills Region': 'cc_12',
                   'Middle Gangetic Plain Region': 'cc_10',
                   'Lower Gangetic Plain Region': 'cc_9',
                   'Western Himalayan Region': 'cc_8',
                   'Eastern Himalayan Region': 'cc_10',
                   'Upper Gangetic Plain Region': 'cc_9',
                   'Trans Gangetic Plain Region': 'cc_9',
                   'Central Plateau & Hills Region': 'cc_7',
                   'Western Plateau and Hills Region': 'cc_11',
                   'Southern Plateau and Hills Region': 'cc_8',
                   'East Coast Plains & Hills Region': 'cc_12'}

In [ ]:
# agroclimatic_zone = 'Eastern Plateau & Hills Region'

# agroclimatic_zone = 'Lower Gangetic Plain Region'

# agroclimatic_zone = 'Middle Gangetic Plain Region'

# agroclimatic_zone = 'Western Himalayan Region'

# agroclimatic_zone = 'Eastern Himalayan Region'

# agroclimatic_zone = 'Upper Gangetic Plain Region'

# agroclimatic_zone = 'Trans Gangetic Plain Region'

# agroclimatic_zone = 'Central Plateau & Hills Region'

# agroclimatic_zone = 'Western Plateau and Hills Region'

agroclimatic_zone = 'Southern Plateau and Hills Region'

# agroclimatic_zone = 'East Coast Plains & Hills Region'

# best_month = best_month_dict[agroclimatic_zone]

## Merge Correction CSVs

In [ ]:
# acz_list = ['Western Himalayan Region', 'Eastern Himalayan Region', 'Lower Gangetic Plain Region',
#             'Middle Gangetic Plain Region', 'Upper Gangetic Plain Region', 'Trans Gangetic Plain Region',
#             'Eastern Plateau & Hills Region', 'Central Plateau & Hills Region', 'Western Plateau and Hills Region',
#             'Southern Plateau and Hills Region', 'East Coast Plains & Hills Region']

acz_list = ['Eastern Plateau & Hills Region']

In [ ]:
year = '2022'

In [ ]:
# CCD

for agroclimatic_zone in acz_list:
    print(agroclimatic_zone)

    df = pd.read_csv(f'drive/MyDrive/harsh/{agroclimatic_zone}.csv')
    dist_list = list(df['Name'])
    print(f'length(dist_list): {len(dist_list)}')

    i = 0
    merged_df = pd.DataFrame()
    for district in dist_list:
        print(i, district)
        i += 1

        try:
            df = pd.read_csv(f'drive/MyDrive/{agroclimatic_zone}/{district}/{year}/result_monthly_cc_corrections.csv')
        except:
            continue

        merged_df = pd.concat([merged_df, df], ignore_index=True)

    print(f'length(merged_df): {len(merged_df)}')

    cols = merged_df.columns
    for i in range(1, len(cols)):
        merged_df[cols[i]] = merged_df[cols[i]].astype('Int64')

    merged_df.to_csv(f'drive/MyDrive/{agroclimatic_zone}/corrections_{year}.csv', index=False)

Western Himalayan Region
length(dist_list): 56
0 Ambala
1 Panchkula
2 Yamunanagar
3 Bilaspur
4 Chamba
5 Hamirpur
6 Kangra
7 Kinnaur
8 Kullu
9 Lahul & Spiti
10 Mandi
11 Shimla
12 Sirmaur
13 Solan
14 Una
15 Anantnag
16 Badgam
17 Bandipore
18 Baramulla
19 Doda
20 Ganderbal
21 Jammu
22 Kargil
23 Kathua
24 Kishtwar
25 Kulgam
26 Kupwara
27 Leh (Ladakh)
28 Poonch
29 Pulwama
30 Rajouri
31 Ramban
32 Reasi
33 Samba
34 Shupiyan
35 Srinagar
36 Udhampur
37 Hoshiarpur
38 Pathankot
39 Rupnagar
40 Sahibzada Ajit Singh Nagar
41 Bijnor
42 Saharanpur
43 Almora
44 Bageshwar
45 Chamoli
46 Champawat
47 Dehradun
48 Garhwal
49 Hardwar
50 Nainital
51 Pithoragarh
52 Rudraprayag
53 Tehri Garhwal
54 Udham Singh Nagar
55 Uttarkashi
length(merged_df): 11147001
Eastern Himalayan Region
length(dist_list): 101
0 Anjaw
1 Changlang
2 Dibang Valley
3 East Kameng
4 East Siang
5 Kurung Kumey
6 Lohit
7 Longding
8 Lower Dibang Valley
9 Lower Subansiri
10 Namsai
11 Papum Pare
12 Tawang
13 Tirap
14 Upper Siang
15 Upper Subansi

In [ ]:
# CH

# Function to convert string representation of list to an actual list
def convert_to_list(string):
    return ast.literal_eval(string)

for agroclimatic_zone in acz_list:
    print(agroclimatic_zone)

    df = pd.read_csv('drive/MyDrive/harsh/district_to_agroclimaticZone_mapping.csv')
    df['IntersectingZones'] = df['IntersectingZones'].apply(convert_to_list)
    district_mapping_df = df[df['AgroclimaticZone'] == agroclimatic_zone][['District', 'IntersectingZones']]
    dist_list = []
    for ind in district_mapping_df.index:
        district = district_mapping_df.loc[ind, 'District']
        zones = district_mapping_df['IntersectingZones'][ind]
        dist_list.append(district)

    print(f'length(dist_list): {len(dist_list)}')

    correction_num = 0
    j = 0
    merged_df = pd.DataFrame()
    for district in dist_list:
        print(j, district)
        j += 1

        try:
            df = pd.read_csv(f'drive/MyDrive/{agroclimatic_zone}/{district}/{year}/result_chm_corrections.csv')
        except:
            continue

        merged_df = pd.concat([merged_df, df], ignore_index=True)
        print(f'length(merged_df): {len(merged_df)}')

        if len(merged_df) > 70000000:
            cols = merged_df.columns
            for i in range(1, len(cols)):
                merged_df[cols[i]] = merged_df[cols[i]].astype('Int64')
            print(f'Saving merged_corrections_{correction_num}.csv')
            merged_df.to_csv(f'drive/MyDrive/{agroclimatic_zone}/corrections_chm_{year}_{correction_num}.csv', index=False)
            del(merged_df)
            correction_num += 1
            merged_df = pd.DataFrame()

    if len(merged_df) > 0:
        cols = merged_df.columns
        for i in range(1, len(cols)):
            merged_df[cols[i]] = merged_df[cols[i]].astype('Int64')
        print(f'Saving merged_corrections_{correction_num}.csv')
        merged_df.to_csv(f'drive/MyDrive/{agroclimatic_zone}/corrections_chm_{year}_{correction_num}.csv', index=False)
        del(merged_df)

## Breaking >10 GB file into smaller files

In [ ]:
df = pd.read_csv(f'drive/MyDrive/{agroclimatic_zone}/Compiled Results/2019/result_0.csv')

In [ ]:
len(df)

84912885

In [ ]:
df1 = df.iloc[0:40000000]

In [ ]:
len(df1)

40000000

In [ ]:
df1.to_csv(f'drive/MyDrive/{agroclimatic_zone}/Compiled Results/2019/result_0_0.csv', index=False)
# df1.to_csv(f'drive/MyDrive/{agroclimatic_zone}/corrections_0.csv', index=False)

In [ ]:
df2 = df.iloc[40000000:]

In [ ]:
len(df2)

44912885

In [ ]:
df2.to_csv(f'drive/MyDrive/{agroclimatic_zone}/Compiled Results/2019/result_0_1.csv', index=False)
# df2.to_csv(f'drive/MyDrive/{agroclimatic_zone}/corrections_1.csv', index=False)

## Combine CSVs CCD


In [ ]:
# Set the years for which results need to be combined
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']

In [ ]:
df = pd.read_csv(f'drive/MyDrive/harsh/{agroclimatic_zone}.csv')
dist_list = list(df['Name'])
len(dist_list)

68

In [ ]:
dist_list

['Chittoor',
 'East Godavari',
 'Guntur',
 'Krishna',
 'Kurnool',
 'Nellore',
 'Prakasam',
 'Srikakulam',
 'Visakhapatnam',
 'Vizianagaram',
 'West Godavari',
 'Y.S.R.',
 'Idukki',
 'Kollam',
 'Pathanamthitta',
 'Thiruvananthapuram',
 'Anugul',
 'Baleshwar',
 'Bauda',
 'Bhadrak',
 'Cuttack',
 'Dhenkanal',
 'Gajapati',
 'Ganjam',
 'Jagatsinghapur',
 'Jajapur',
 'Kandhamal',
 'Kendrapara',
 'Kendujhar',
 'Khordha',
 'Koraput',
 'Malkangiri',
 'Mayurbhanj',
 'Nayagarh',
 'Puri',
 'Rayagada',
 'Karaikal',
 'Puducherry',
 'Yanam',
 'Ariyalur',
 'Chennai',
 'Cuddalore',
 'Dharmapuri',
 'Kancheepuram',
 'Kanniyakumari',
 'Krishnagiri',
 'Madurai',
 'Nagappattinam',
 'Pudukkottai',
 'Ramanathapuram',
 'Salem',
 'Sivaganga',
 'Thanjavur',
 'Theni',
 'Thiruvallur',
 'Thiruvarur',
 'Thoothukkudi',
 'Tiruchirappalli',
 'Tirunelveli',
 'Tiruvannamalai',
 'Vellore',
 'Viluppuram',
 'Virudunagar',
 'Khammam',
 'Mahbubnagar',
 'Nalgonda',
 'Pashchim Medinipur',
 'Purba Medinipur']

In [ ]:
for year in years:
  print(year)
  df_con = pd.DataFrame()
  df_len = 0
  result_num = 0
  dst_num = 0
  for district in dist_list:
    print(dst_num, district)
    dst_num += 1
    try:
        df = pd.read_csv(f'drive/MyDrive/{agroclimatic_zone}/{district}/{year}/result_monthly_cc.csv')
    except:
        continue
    df_len += len(df)
    df_con = pd.concat([df_con, df])
    del(df)
    print(df_len)

    if df_len > 80000000:
      print(f'Saving result_{result_num}.csv')
      df_con['cc'] = df_con[best_month]
      path = f'drive/MyDrive/{agroclimatic_zone}/Compiled Results/{year}/'
      if not os.path.exists(path):
        os.makedirs(path)
      df_con.to_csv(f'drive/MyDrive/{agroclimatic_zone}/Compiled Results/{year}/result_{result_num}.csv', index=False)
      result_num += 1
      del(df_con)
      df_len = 0
      df_con = pd.DataFrame()

  if (len(df_con) > 0):
    print(f'Saving result_{result_num}.csv')
    df_con['cc'] = df_con[best_month]
    path = f'drive/MyDrive/{agroclimatic_zone}/Compiled Results/{year}/'
    if not os.path.exists(path):
        os.makedirs(path)
    df_con.to_csv(f'drive/MyDrive/{agroclimatic_zone}/Compiled Results/{year}/result_{result_num}.csv', index=False)
    result_num += 1
    del(df_con)
    df_len = 0
    df_con = pd.DataFrame()

2016
0 Chittoor
910279
1 East Godavari
911372
2 Guntur
3237135
3 Krishna
6780571
4 Kurnool
6781309
5 Nellore
7378183
6 Prakasam
12042722
7 Srikakulam
8 Visakhapatnam
13887608
9 Vizianagaram
14333636
10 West Godavari
14990204
11 Y.S.R.
14991427
12 Idukki
13 Kollam
14 Pathanamthitta
15 Thiruvananthapuram
16 Anugul
14995129
17 Baleshwar
16673737
18 Bauda
19 Bhadrak
17229815
20 Cuttack
19490365
21 Dhenkanal
19581884
22 Gajapati
23 Ganjam
24 Jagatsinghapur
19935851
25 Jajapur
21251690
26 Kandhamal
27 Kendrapara
21551187
28 Kendujhar
21593568
29 Khordha
23197141
30 Koraput
31 Malkangiri
23201504
32 Mayurbhanj
23220045
33 Nayagarh
24526302
34 Puri
25480595
35 Rayagada
36 Karaikal
37 Puducherry
38 Yanam
39 Ariyalur
25485736
40 Chennai
41 Cuddalore
25582576
42 Dharmapuri
43 Kancheepuram
25582577
44 Kanniyakumari
45 Krishnagiri
46 Madurai
47 Nagappattinam
48 Pudukkottai
25612978
49 Ramanathapuram
25615561
50 Salem
25622763
51 Sivaganga
25763316
52 Thanjavur
25804074
53 Theni
54 Thiruvallur
26123

## Combine CSVs CH

In [ ]:
# agroclimatic_zone = 'Eastern Plateau & Hills Region'
# agroclimatic_zone = 'Southern Plateau and Hills Region'
# agroclimatic_zone = 'East Coast Plains & Hills Region'
# agroclimatic_zone = 'Western Plateau and Hills Region'
# agroclimatic_zone = 'Central Plateau & Hills Region'
# agroclimatic_zone = 'Lower Gangetic Plain Region'
# agroclimatic_zone = 'Middle Gangetic Plain Region'
# agroclimatic_zone = 'Trans Gangetic Plain Region'
# agroclimatic_zone = 'Upper Gangetic Plain Region'
# agroclimatic_zone = 'Western Himalayan Region'
agroclimatic_zone = 'Eastern Himalayan Region'

In [ ]:
df = pd.read_csv('drive/MyDrive/harsh/district_to_agroclimaticZone_mapping.csv')

In [ ]:
# Function to convert string representation of list to an actual list
def convert_to_list(string):
    return ast.literal_eval(string)

df['IntersectingZones'] = df['IntersectingZones'].apply(convert_to_list)

In [ ]:
district_mapping_df = df[df['AgroclimaticZone'] == agroclimatic_zone][['District', 'IntersectingZones']]

In [ ]:
dist_list = []
i = 0
for ind in district_mapping_df.index:
    district = district_mapping_df.loc[ind, 'District']
    zones = district_mapping_df['IntersectingZones'][ind]
    print(i, district, zones)
    dist_list.append(district)
    i += 1

0 Anjaw ['Eastern Himalayan Region']
1 Changlang ['Eastern Himalayan Region']
2 Dibang Valley ['Eastern Himalayan Region']
3 East Kameng ['Eastern Himalayan Region']
4 East Siang ['Eastern Himalayan Region']
5 Kurung Kumey ['Eastern Himalayan Region']
6 Lohit ['Eastern Himalayan Region']
7 Longding ['Eastern Himalayan Region']
8 Lower Dibang Valley ['Eastern Himalayan Region']
9 Lower Subansiri ['Eastern Himalayan Region']
10 Namsai ['Eastern Himalayan Region']
11 Papum Pare ['Eastern Himalayan Region']
12 Tawang ['Eastern Himalayan Region']
13 Tirap ['Eastern Himalayan Region']
14 Upper Siang ['Eastern Himalayan Region']
15 Upper Subansiri ['Eastern Himalayan Region']
16 West Kameng ['Eastern Himalayan Region']
17 West Siang ['Eastern Himalayan Region']
18 Baksa ['Eastern Himalayan Region']
19 Barpeta ['Eastern Himalayan Region']
20 Bongaigaon ['Eastern Himalayan Region']
21 Cachar ['Eastern Himalayan Region']
22 Chirang ['Eastern Himalayan Region']
23 Darrang ['Eastern Himalayan Regi

In [ ]:
years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022']

In [ ]:
for year in years:
  print(year)
  df_con = pd.DataFrame()
  df_len = 0
  result_num = 0
  dst_num = 0
  for district in dist_list:
    print(dst_num, district)
    dst_num += 1
    try:
        df = pd.read_csv(f'drive/MyDrive/{agroclimatic_zone}/{district}/{year}/result_chm.csv')
    except:
        continue
    df_len += len(df)
    df_con = pd.concat([df_con, df])
    del(df)
    print(df_len)

    if df_len > 80000000:
        print(f'Saving result_{result_num}.csv')

        conditions = [
            (df_con['rh50_class'] == 0) & (df_con['rh75_class'] == 0) & (df_con['rh98_class'] == 0),
            (df_con['rh50_class'] == 0) & (df_con['rh75_class'] == 0) & (df_con['rh98_class'] == 1),
            (df_con['rh50_class'] == 0) & (df_con['rh75_class'] == 1) & (df_con['rh98_class'] == 0),
            (df_con['rh50_class'] == 0) & (df_con['rh75_class'] == 1) & (df_con['rh98_class'] == 1),
            (df_con['rh50_class'] == 1) & (df_con['rh75_class'] == 0) & (df_con['rh98_class'] == 0),
            (df_con['rh50_class'] == 1) & (df_con['rh75_class'] == 0) & (df_con['rh98_class'] == 1)
        ]

        choices = [0, 0, 1, 2, 1, 2]

        df_con['ch_class'] = np.select(conditions, choices, default=3)

        path = f'drive/MyDrive/{agroclimatic_zone}/Compiled Results CH/{year}/'
        if not os.path.exists(path):
            os.makedirs(path)
        df_con.to_csv(f'drive/MyDrive/{agroclimatic_zone}/Compiled Results CH/{year}/result_{result_num}.csv', index=False)
        result_num += 1
        del(df_con)
        df_len = 0
        df_con = pd.DataFrame()

  if (len(df_con) > 0):
    print(f'Saving result_{result_num}.csv')

    conditions = [
        (df_con['rh50_class'] == 0) & (df_con['rh75_class'] == 0) & (df_con['rh98_class'] == 0),
        (df_con['rh50_class'] == 0) & (df_con['rh75_class'] == 0) & (df_con['rh98_class'] == 1),
        (df_con['rh50_class'] == 0) & (df_con['rh75_class'] == 1) & (df_con['rh98_class'] == 0),
        (df_con['rh50_class'] == 0) & (df_con['rh75_class'] == 1) & (df_con['rh98_class'] == 1),
        (df_con['rh50_class'] == 1) & (df_con['rh75_class'] == 0) & (df_con['rh98_class'] == 0),
        (df_con['rh50_class'] == 1) & (df_con['rh75_class'] == 0) & (df_con['rh98_class'] == 1)
    ]

    choices = [0, 0, 1, 2, 1, 2]

    df_con['ch_class'] = np.select(conditions, choices, default=3)

    path = f'drive/MyDrive/{agroclimatic_zone}/Compiled Results CH/{year}/'
    if not os.path.exists(path):
        os.makedirs(path)
    df_con.to_csv(f'drive/MyDrive/{agroclimatic_zone}/Compiled Results CH/{year}/result_{result_num}.csv', index=False)
    result_num += 1
    del(df_con)
    df_len = 0
    df_con = pd.DataFrame()

2016
0 Anjaw
1 Changlang
2 Dibang Valley
3 East Kameng
1507699
4 East Siang
5 Kurung Kumey
6 Lohit
7 Longding
8 Lower Dibang Valley
1507718
9 Lower Subansiri
10 Namsai
11 Papum Pare
12 Tawang
13 Tirap
14 Upper Siang
15 Upper Subansiri
16 West Kameng
3493417
17 West Siang
18 Baksa
5005946
19 Barpeta
5321898
20 Bongaigaon
5380011
21 Cachar
22 Chirang
5500865
23 Darrang
5959199
24 Dhemaji
25 Dhubri
7190677
26 Dibrugarh
27 Dima Hasao
28 Goalpara
8625990
29 Golaghat
30 Hailakandi
8626192
31 Jorhat
32 Kamrup Metropolitan
8999843
33 Kamrup
9802972
34 Karbi Anglong
10167776
35 Karimganj
10707737
36 Kokrajhar
13218216
37 Lakhimpur
38 Morigaon
13679346
39 Nagaon
14904449
40 Nalbari
15362965
41 Sivasagar
42 Sonitpur
17647470
43 Tinsukia
44 Udalguri
19036929
45 Bishnupur
46 Chandel
47 Churachandpur
48 Imphal East
49 Imphal West
50 Senapati
51 Tamenglong
52 Thoubal
53 Ukhrul
19245668
54 East Garo Hills
23031255
55 East Khasi Hills
24586957
56 Jaintia Hills
24808228
57 North Garo Hills
25812383
58 R